In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import re
import collections
import random
from time import time

from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, FastICA

import data_handler as dh
import semeval_data_helper as sdh


# plot settings
% matplotlib inline
# print(plt.rcParams.keys())
# plt.rcParams['figure.figsize'] = (16,9)

import mpld3

In [2]:
# reload(eh)
import experiment_helper as eh

In [3]:
shuffle_seed = 20

In [7]:
reload(dh)
DH = dh.DataHandler('data/semeval_train_sdp__include_8000', valid_percent=10, shuffle_seed=shuffle_seed) # for semeval

Creating Data objects...
Done creating Data objects
7999 total examples :: 7199 training : 800 valid (90:10 split)
Vocab size: 22683 Dep size: 50 POS size: 18


In [9]:
# reload(sdh)
train, valid, test, label2int, int2label = sdh.load_semeval_data(include_ends=True, shuffle_seed=shuffle_seed)
num_classes = len(int2label.keys())

Didn't find common ancestor
1790	"The imams were removed from a US Airways <e1>flight</e1> awaiting departure from the Minneapolis-St. Paul <e2>airport</e2>."

(The imams were removed from a US Airways flight awaiting departure from the Minneapolis - St . Paul airport ., flight , airport )
Bad sentence: '1790\t"The imams were removed from a US Airways <e1>flight</e1> awaiting departure from the Minneapolis-St. Paul <e2>airport</e2>."\r\n'
((The imams were removed from a US Airways flight awaiting departure from the Minneapolis - St . Paul airport ., flight , airport ), None)
Skipping this one... '1790\t"The imams were removed from a US Airways <e1>flight</e1> awaiting departure from the Minneapolis-St. Paul <e2>airport</e2>."\r\n'
(None, None, None, 4)
Num training: 7199
Num valididation: 800
Didn't find common ancestor
8310	"Tributes have been paid to the <e1>writer</e1> who created Goodness Gracious Me, the hit BBC television <e2>series</e2>."

(Tributes have been paid to the writer 

In [ ]:
### WE DONT WANT INDICES THIS TIME ###
# # convert the semeval data to indices under the wiki vocab:
# train['sdps'] = DH.sentences_to_sequences(train['sdps'])
# valid['sdps'] = DH.sentences_to_sequences(valid['sdps'])
# test['sdps'] = DH.sentences_to_sequences(test['sdps'])
    
# train['targets'] = DH.sentences_to_sequences(train['targets'])
# valid['targets'] = DH.sentences_to_sequences(valid['targets'])
# test['targets'] = DH.sentences_to_sequences(test['targets'])

# print(train['targets'][:5]) # small sample

In [ ]:
# max_seq_len = max([len(path) for path in train['sdps']+valid['sdps']+test['sdps']])
# print(max_seq_len, DH.max_seq_len)
# DH.max_seq_len = max_seq_len

# Simple Logistic Regression on the SDPs w/ TFIDF

In [14]:
from sklearn.pipeline import Pipeline
# define baseline pipelines
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Feature Extractors
cv = CountVectorizer(
        input=u'content', 
        encoding=u'utf-8', 
        decode_error=u'strict', 
        strip_accents='unicode', 
        lowercase=True,
        analyzer=u'word', 
        preprocessor=None, 
        tokenizer=None, 
        stop_words='english', 
        #token_pattern=u'(?u)\\b\w\w+\b', # one alphanumeric is a token
        ngram_range=(1, 2), 
        max_df=.9, 
        min_df=2, 
        max_features=None, 
        vocabulary=None, 
        binary=False, 
        #dtype=type 'numpy.int64'>
        )
from sklearn.feature_extraction.text import TfidfTransformer
tf = TfidfTransformer(
        norm='l2',
        use_idf=True,
        smooth_idf=True,
        sublinear_tf=False
)

# Final Classifier
lr = LogisticRegression(C=.05,
                        fit_intercept=True,
                        random_state=0,
                        class_weight='balanced',
#                         multi_class='multinomial',
                        #solver='lbfgs',
                        n_jobs=1)

pipeline = Pipeline([
    ('count', cv),
    ('tfidf', tf),
    ('logreg', lr)
    ])

param_grid = {
    'count__ngram_range':[(1,1),(1,2),(1,3)],
    'tfidf__norm':['l1', 'l2'],
    'tfidf__use_idf':[True, False],
    'tfidf__sublinear_tf':[True,False],
    'logreg__C':[.001, .01, .1],
    'logreg__penalty':['l1', 'l2']
}

from sklearn.grid_search import GridSearchCV
grid_search = GridSearchCV(pipeline, 
                           param_grid,
                           scoring='f1_macro',
                           n_jobs=-1, verbose=1)

print("Here")
x_data = [sent[0].text for sent in train['sents']]
y_data = train['labels']
print(x_data[0], y_data[0])
grid_search.fit(np.array(x_data), y_data)
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    

Here
Tom plans to continue to live the country life as he grows older . 18
Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] count__ngram_range=(1, 1), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l1, tfidf__use_idf=True 
[CV] count__ngram_range=(1, 1), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l1, tfidf__use_idf=True 
[CV] count__ngram_range=(1, 1), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l1, tfidf__use_idf=True 
[CV] count__ngram_range=(1, 1), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l1, tfidf__use_idf=False 


/Users/thomaseffland/.virtualenvs/rel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  count__ngram_range=(1, 1), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l1, tfidf__use_idf=True -   0.7s
[CV] count__ngram_range=(1, 1), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l1, tfidf__use_idf=False 


/Users/thomaseffland/.virtualenvs/rel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  count__ngram_range=(1, 1), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l1, tfidf__use_idf=True -   0.8s
[CV] count__ngram_range=(1, 1), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l1, tfidf__use_idf=False 


...

[CV]  count__ngram_range=(1, 2), tfidf__sublinear_tf=False, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l1, tfidf__use_idf=False -   0.9s
[CV] count__ngram_range=(1, 2), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l2, tfidf__use_idf=True 
[CV] count__ngram_range=(1, 2), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l2, tfidf__use_idf=False 
[CV]  count__ngram_range=(1, 2), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l2, tfidf__use_idf=True -   0.9s
[CV] count__ngram_range=(1, 2), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l2, tfidf__use_idf=False 
[CV]  count__ngram_range=(1, 2), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l2, tfidf__use_idf=True -   0.9s
[CV] count__ngram_range=(1, 2), tfidf__sublinear_tf=True, logreg__penalty=l1, logreg__C=0.001, tfidf__norm=l2, tfidf__use_idf=False 
[CV]  count__ngram_range=(1, 2), tfidf__subl

[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.8min


[CV] count__ngram_range=(1, 3), tfidf__sublinear_tf=True, logreg__penalty=l2, logreg__C=0.01, tfidf__norm=l1, tfidf__use_idf=True 
[CV]  count__ngram_range=(1, 3), tfidf__sublinear_tf=False, logreg__penalty=l1, logreg__C=0.01, tfidf__norm=l2, tfidf__use_idf=False -   1.4s
[CV] count__ngram_range=(1, 3), tfidf__sublinear_tf=True, logreg__penalty=l2, logreg__C=0.01, tfidf__norm=l1, tfidf__use_idf=True 
[CV]  count__ngram_range=(1, 3), tfidf__sublinear_tf=False, logreg__penalty=l1, logreg__C=0.01, tfidf__norm=l2, tfidf__use_idf=False -   1.4s
[CV] count__ngram_range=(1, 3), tfidf__sublinear_tf=True, logreg__penalty=l2, logreg__C=0.01, tfidf__norm=l1, tfidf__use_idf=False 
[CV]  count__ngram_range=(1, 3), tfidf__sublinear_tf=True, logreg__penalty=l2, logreg__C=0.01, tfidf__norm=l1, tfidf__use_idf=True -   1.3s
[CV] count__ngram_range=(1, 3), tfidf__sublinear_tf=True, logreg__penalty=l2, logreg__C=0.01, tfidf__norm=l1, tfidf__use_idf=False 
[CV]  count__ngram_range=(1, 3), tfidf__sublinear_

[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  2.2min finished



Best score: 0.328
Best parameters set:
	count__ngram_range: (1, 1)
	logreg__C: 0.1
	logreg__penalty: 'l2'
	tfidf__norm: 'l2'
	tfidf__sublinear_tf: True
	tfidf__use_idf: False


In [17]:
from sklearn.metrics import f1_score
test_x = [sent[0].text for sent in valid['sents']]
test_y = valid['labels']

In [20]:
lr_sent = grid_search.best_estimator_
lr_sent.fit(x_data, y_data)
preds = lr_sent.predict(test_x)
lr_sent_score = f1_score(test_y, preds, average='macro')
print('LR Sentence validation f1 macro: %2.4f' % lr_sent_score)

LR Sentence validation f1 macro: 0.3291


/Users/thomaseffland/.virtualenvs/rel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# Logistic Regression on the words in SDPs

In [22]:
from sklearn.pipeline import Pipeline
# define baseline pipelines
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Feature Extractors
cv = CountVectorizer(
        input=u'content', 
        encoding=u'utf-8', 
        decode_error=u'strict', 
        strip_accents='unicode', 
        lowercase=True,
        analyzer=u'word', 
        preprocessor=None, 
        tokenizer=None, 
        stop_words='english', 
        #token_pattern=u'(?u)\\b\w\w+\b', # one alphanumeric is a token
        ngram_range=(1, 2), 
        max_df=.9, 
        min_df=2, 
        max_features=None, 
        vocabulary=None, 
        binary=False, 
        #dtype=type 'numpy.int64'>
        )
from sklearn.feature_extraction.text import TfidfTransformer
tf = TfidfTransformer(
        norm='l2',
        use_idf=True,
        smooth_idf=True,
        sublinear_tf=False
)

# Final Classifier
lr = LogisticRegression(C=.05,
                        fit_intercept=True,
                        random_state=0,
                        class_weight='balanced',
#                         multi_class='multinomial',
                        #solver='lbfgs',
                        n_jobs=1)

pipeline = Pipeline([
    ('count', cv),
    ('tfidf', tf),
    ('logreg', lr)
    ])

param_grid = {
    'count__ngram_range':[(1,1),(1,2),(1,3)],
    'tfidf__norm':['l1', 'l2'],
    'tfidf__use_idf':[True, False],
    'tfidf__sublinear_tf':[True,False],
    'logreg__C':[.001, .01, .1],
    'logreg__penalty':['l1', 'l2']
}

from sklearn.grid_search import GridSearchCV
grid_search = GridSearchCV(pipeline, 
                           param_grid,
                           scoring='f1_macro',
                           n_jobs=-1, verbose=1)

print("Here")
x_data = [" ".join([tok[0] for tok in sdp]) for sdp in train['sdps']]
y_data = train['labels']
print(x_data[0], y_data[0])
grid_search.fit(np.array(x_data), y_data)
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    

Here
country life 18
Fitting 3 folds for each of 144 candidates, totalling 432 fits


/Users/thomaseffland/.virtualenvs/rel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thomaseffland/.virtualenvs/rel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thomaseffland/.virtualenvs/rel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thomaseffland/.virtualenvs/rel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 


Best score: 0.431
Best parameters set:
	count__ngram_range: (1, 3)
	logreg__C: 0.1
	logreg__penalty: 'l2'
	tfidf__norm: 'l2'
	tfidf__sublinear_tf: False
	tfidf__use_idf: True


In [23]:
lr_sdp = grid_search.best_estimator_
lr_sdp.fit(x_data, y_data)
preds = lr_sdp.predict(test_x)
lr_sdp_score = f1_score(test_y, preds, average='macro')
print('LR SDP validation f1 macro: %2.4f' % lr_sdp_score)

LR SDP validation f1 macro: 0.3663


# Logistic Regression on SDP with Brown clusters bag

In [49]:
brown_clusters = {}
lines = open('data/shuffled.en.tok-c50-p1.out/paths', 'r').readlines()
for line in lines:
    vec = line.split()
    brown_clusters[vec[1]] = vec[0]
del lines

In [50]:
# append brown clusters to each sdp sentence
x_data = [" ".join([tok[0] for tok in sdp]) + " " + 
          " ".join([brown_clusters[str(tok[0])] for tok in sdp if str(tok[0]) in brown_clusters])
          for sdp in train['sdps']]
print(x_data[:5])
test_x = [" ".join([tok[0] for tok in sdp]) + " " + 
          " ".join([brown_clusters[str(tok[0])] for tok in sdp if str(tok[0]) in brown_clusters])
          for sdp in valid['sdps']]
x_data_ends = [" ".join([tok[0] for tok in sdp]) + " " + 
          " ".join([brown_clusters[str(tok[0])] for tok in [sdp[0], sdp[-1]] if str(tok[0]) in brown_clusters])
          for sdp in train['sdps']]
test_x_ends = [" ".join([tok[0] for tok in sdp]) + " " + 
          " ".join([brown_clusters[str(tok[0])] for tok in [sdp[0], sdp[-1]] if str(tok[0]) in brown_clusters])
          for sdp in valid['sdps']]

[u'country life 011011 011011', u'car has player casette 011011 01111111110 011011 000110', u'jam of women 011011 00100 011011', u'view is from bridge 011011 0111111100 0010111 011011', u'hill of corn 011011 00100 0111011']


In [52]:
from sklearn.pipeline import Pipeline
# define baseline pipelines
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Feature Extractors
cv = CountVectorizer(
        input=u'content', 
        encoding=u'utf-8', 
        decode_error=u'strict', 
        strip_accents='unicode', 
        lowercase=True,
        analyzer=u'word', 
        preprocessor=None, 
        tokenizer=None, 
        stop_words='english', 
        #token_pattern=u'(?u)\\b\w\w+\b', # one alphanumeric is a token
        ngram_range=(1, 2), 
        max_df=.9, 
        min_df=2, 
        max_features=None, 
        vocabulary=None, 
        binary=False, 
        #dtype=type 'numpy.int64'>
        )
from sklearn.feature_extraction.text import TfidfTransformer
tf = TfidfTransformer(
        norm='l2',
        use_idf=True,
        smooth_idf=True,
        sublinear_tf=False
)

# Final Classifier
lr = LogisticRegression(C=.05,
                        fit_intercept=True,
                        random_state=0,
                        class_weight='balanced',
#                         multi_class='multinomial',
                        #solver='lbfgs',
                        n_jobs=1)

pipeline = Pipeline([
    ('count', cv),
    ('tfidf', tf),
    ('logreg', lr)
    ])

param_grid = {
    'count__ngram_range':[(1,1),(1,2),(1,3)],
    'tfidf__norm':['l1', 'l2'],
    'tfidf__use_idf':[True, False],
    'tfidf__sublinear_tf':[True,False],
    'logreg__C':[.001, .01, .1, .5, 1.],
    'logreg__penalty':['l1', 'l2']
}

from sklearn.grid_search import GridSearchCV
grid_search = GridSearchCV(pipeline, 
                           param_grid,
                           scoring='f1_macro',
                           n_jobs=-1, verbose=1)

print(x_data[0], y_data[0])
grid_search.fit(np.array(x_data), y_data)
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    

country life 011011 011011 18
Fitting 3 folds for each of 240 candidates, totalling 720 fits


/Users/thomaseffland/.virtualenvs/rel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thomaseffland/.virtualenvs/rel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thomaseffland/.virtualenvs/rel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thomaseffland/.virtualenvs/rel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 


Best score: 0.550
Best parameters set:
	count__ngram_range: (1, 1)
	logreg__C: 1.0
	logreg__penalty: 'l2'
	tfidf__norm: 'l2'
	tfidf__sublinear_tf: False
	tfidf__use_idf: True


In [53]:
lr_sdp_brown = grid_search.best_estimator_
lr_sdp_brown.fit(x_data, y_data)
preds = lr_sdp_brown.predict(test_x)
lr_sdp_brown_score = f1_score(test_y, preds, average='macro')
print('LR SDP validation f1 macro: %2.4f' % lr_sdp_brown_score)

LR SDP validation f1 macro: 0.5969


In [47]:
test_x = [" ".join([tok[0] for tok in sdp]) + " " + 
          " ".join([brown_clusters[str(tok[0])] for tok in sdp if str(tok[0]) in brown_clusters])
          for sdp in test['sdps']]

preds = lr_sdp_brown.predict(test_x)
with open('SemEval2010_task8_all_data/test_pred.txt', 'w') as f:
    i = 8001
    for pred in preds:
        f.write("%i\t%s\n" % (i, int2label[pred]))
        i += 1

In [48]:
%%bash
./SemEval2010_task8_all_data/SemEval2010_task8_scorer-v1.2/semeval2010_task8_scorer-v1.2.pl \
SemEval2010_task8_all_data/test_pred.txt SemEval2010_task8_all_data/test_keys.txt

<<< (2*9+1)-WAY EVALUATION (USING DIRECTIONALITY)>>>:

Confusion matrix:
        C-E1 C-E2 C-W1 C-W2 C-C1 C-C2 E-D1 E-D2 E-O1 E-O2 I-A1 I-A2 M-C1 M-C2 M-T1 M-T2 P-P1 P-P2  _O_ <-- classified as
      +-----------------------------------------------------------------------------------------------+ -SUM- skip ACTUAL
 C-E1 |  72   39    2    0    0    0    2    0    0    1    0    1    0   11    1    1    2    0    2 |  134    0  134
 C-E2 |   5  153    0    1    0    0    0    0    4    1    0    1    0    6    3    0    3    1   16 |  194    0  194
 C-W1 |   1    0   71   13    5    2    4    0    4    0    3    1    3   20    6    0    5    1   23 |  162    0  162
 C-W2 |   0    3   20   54    1    0    6    0    2    2    1   10    1   28    3    0    3    4   12 |  150    0  150
 C-C1 |   0    1    4    5   89   20   14    0    2    4    0    0    2    2    0    0    0    0   10 |  153    0  153
 C-C2 |   0    0    1    0    3   23    1    0    0    1    0    1    1    2    1    0   